In [1]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf, SQLContext

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import os

CLICKHOUSE_IP = "34.32.60.106"
CLICKHOUSE_PORT = 9000
CLICKHOUSE_USER = "default"
CLICKHOUSE_USER_PASSWORD = "1278"

#https://repo1.maven.org/maven2/com/github/housepower/clickhouse-native-jdbc/2.7.1/clickhouse-native-jdbc-2.7.1.jar
packages = [
    "com.github.housepower:clickhouse-spark-runtime-3.4_2.12:0.7.3",
    "com.clickhouse:clickhouse-jdbc:0.6.0-patch4",
    "com.clickhouse:clickhouse-http-client:0.6.0-patch4",
    "org.apache.httpcomponents.client5:httpclient5:5.3.1",
    "com.github.housepower:clickhouse-native-jdbc:2.7.1"
]
ram = 30
cpu = 22*3
# Define the application name and setup session
appName = "Connect To ClickHouse via PySpark"
spark = (SparkSession.builder
         .appName(appName)
         .config("spark.jars.packages", ",".join(packages))
         .config("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
         .config("spark.sql.catalog.clickhouse.host", CLICKHOUSE_IP)
         .config("spark.sql.catalog.clickhouse.protocol", "http")
         .config("spark.sql.catalog.clickhouse.http_port", "8123")
         .config("spark.sql.catalog.clickhouse.user", CLICKHOUSE_USER)
         .config("spark.sql.catalog.clickhouse.password", CLICKHOUSE_USER_PASSWORD)
         .config("spark.sql.catalog.clickhouse.database", "default")
         #.config("spark.spark.clickhouse.write.compression.codec", "lz4")
         #.config("spark.clickhouse.read.compression.codec", "lz4")
         #.config("spark.clickhouse.write.format", "arrow")
         #    .config("spark.clickhouse.write.distributed.convertLocal", "true") 
         #    .config("spark.clickhouse.write.repartitionNum", "1") 
         #.config("spark.clickhouse.write.maxRetry", "1000")
         #    .config("spark.clickhouse.write.repartitionStrictly", "true") 
         #    .config("spark.clickhouse.write.distributed.useClusterNodes", "false") 
         #.config("spark.clickhouse.write.batchSize", "1000000")
         #.config("spark.sql.catalog.clickhouse.socket_timeout", "600000000")
         #  .config("spark.sql.catalog.clickhouse.connection_timeout", "600000000")
         #  .config("spark.sql.catalog.clickhouse.query_timeout", "600000000")
         #  .config("spark.clickhouse.options.socket_timeout", "600000000")
         #  .config("spark.clickhouse.options.connection_timeout", "600000000")
         #  .config("spark.clickhouse.options.query_timeout", "600000000")         
         .config("spark.executor.memory", f"{ram}g")
         #.config("spark.executor.cores", "5")
         .config("spark.driver.maxResultSize", f"{ram}g")
         #.config("spark.driver.memory", f"{ram}g")
         #.config("spark.executor.memoryOverhead", f"{ram}g")
         #.config("spark.sql.debug.maxToStringFields", "100000")
         .getOrCreate()
         )
#SedonaRegistrator.registerAll(spark)
# spark.conf.set("spark.sql.catalog.clickhouse", "xenon.clickhouse.ClickHouseCatalog")
# spark.conf.set("spark.sql.catalog.clickhouse.host", "127.0.0.1")
# spark.conf.set("spark.sql.catalog.clickhouse.protocol", "http")
# spark.conf.set("spark.sql.catalog.clickhouse.http_port", "8123")
# spark.conf.set("spark.sql.catalog.clickhouse.user", "default")
# spark.conf.set("spark.sql.catalog.clickhouse.password", "")
# spark.conf.set("spark.sql.catalog.clickhouse.database", "default")
spark.sql("use clickhouse")

from ydata_profiling import ProfileReport
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import mlflow
import optuna


root_path = "/app"
#root_path = "."
#path_data = f'{root_path}/data'
your_mlflow_tracking_uri = f'{root_path}/mlruns'

mlflow.set_tracking_uri(your_mlflow_tracking_uri)

import warnings
warnings.filterwarnings("ignore")

from optuna.integration.mlflow import MLflowCallback





:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.github.housepower#clickhouse-spark-runtime-3.4_2.12 added as a dependency
com.clickhouse#clickhouse-jdbc added as a dependency
com.clickhouse#clickhouse-http-client added as a dependency
org.apache.httpcomponents.client5#httpclient5 added as a dependency
com.github.housepower#clickhouse-native-jdbc added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-57fa3c0e-22be-4fff-8df8-60865bb400fc;1.0
	confs: [default]
	found com.github.housepower#clickhouse-spark-runtime-3.4_2.12;0.7.3 in central
	found com.clickhouse#clickhouse-jdbc;0.6.0-patch4 in central
	found com.clickhouse#clickhouse-http-client;0.6.0-patch4 in central
	found com.clickhouse#clickhouse-client;0.6.0-patch4 in central
	found com.clickhouse#clickhouse-data;0.6.0-patch4 in central
	found org.apache.httpcomponents.client5#httpclient5;5.3.1 in central
	found org.apache.httpcomponents.core5#httpc

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
import sklearn
sklearn.set_config(transform_output="pandas")
from sklearn.utils import compute_sample_weight
#
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import Ridge
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
import seaborn as sns


from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import RobustScaler

from sklearn.cluster import KMeans

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.svm import SVC

import xgboost as xgb
from xgboost import XGBRegressor, XGBClassifier

from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer

import optuna

import mlflow

import warnings

warnings.simplefilter('ignore')

import joblib

from joblib import dump, load

import requests

client = mlflow.MlflowClient(tracking_uri = "/app/mlruns")

mlflow.set_tracking_uri("/app/mlruns")

import requests

from sklearn import impute

from sklearn.impute import KNNImputer

from sklearn.experimental import enable_iterative_imputer

from sklearn.impute import IterativeImputer

from imblearn.over_sampling import SMOTE

from imblearn.pipeline import Pipeline

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import precision_recall_curve, average_precision_score

In [3]:
data = spark.read.csv('files/train.csv', header=True, inferSchema=True)
data.show()

AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/app/files/train.csv.

In [2]:
import clickhouse_connect
import pandas as pd

# Define your ClickHouse connection details
CLICKHOUSE_IP = "34.32.60.106"
CLICKHOUSE_USER = "default"
CLICKHOUSE_USER_PASSWORD = "1278"
DATABASE = "bank"
TABLE = "salaryincome"

# Create a ClickHouse client
client = clickhouse_connect.get_client(host=CLICKHOUSE_IP,
                                       username=CLICKHOUSE_USER,
                                       password=CLICKHOUSE_USER_PASSWORD)

# Load data
data = pd.read_csv('/opt/spark/work-dir/files/train.csv')

# Rename columns to avoid issues with special characters
data.rename(columns={
    'educational-num': 'educational_num',
    'marital-status': 'marital_status',
    'capital-gain': 'capital_gain',
    'capital-loss': 'capital_loss',
    'hours-per-week': 'hours_per_week',
    'native-country': 'native_country',
    'income_>50K': 'income_50K'
}, inplace=True)

# Define a mapping for categorical columns to integer values
categorical_columns = [
    'workclass', 'education', 'marital_status', 'occupation',
    'relationship', 'race', 'gender', 'native_country'
]

# Convert categorical columns to integers
for column in categorical_columns:
    data[column] = data[column].astype('category').cat.codes

# Convert remaining columns to appropriate integer types
for column in data.columns:
    if column == 'age':
        data[column] = data[column].astype('uint8')
    else:
        data[column] = data[column].astype('uint32')

# Define the ClickHouse table schema if not already defined
client.command(f"""
CREATE TABLE IF NOT EXISTS {DATABASE}.{TABLE} (
    age UInt8,
    workclass UInt32,
    fnlwgt UInt32,
    education UInt32,
    educational_num UInt32,
    marital_status UInt32,
    occupation UInt32,
    relationship UInt32,
    race UInt32,
    gender UInt32,
    capital_gain UInt32,
    capital_loss UInt32,
    hours_per_week UInt32,
    native_country UInt32,
    income_50K UInt32 
) ENGINE = MergeTree()
ORDER BY income_50K
""")

# Convert DataFrame to a list of tuples for insertion
data_tuples = [tuple(x) for x in data.to_numpy()]

# Insert data into ClickHouse
client.insert(f'{DATABASE}.{TABLE}', data)

# Close the ClickHouse client
client.close()


In [5]:
df = client.query('SELECT * FROM bank.salaryincome')
df.result_columns

[[26,
  37,
  18,
  28,
  42,
  27,
  28,
  51,
  46,
  42,
  48,
  65,
  27,
  37,
  35,
  28,
  34,
  36,
  50,
  60,
  40,
  35,
  40,
  29,
  64,
  71,
  37,
  41,
  40,
  26,
  20,
  32,
  48,
  36,
  73,
  48,
  45,
  20,
  55,
  22,
  49,
  56,
  50,
  23,
  37,
  36,
  72,
  64,
  54,
  46,
  32,
  46,
  65,
  49,
  45,
  30,
  72,
  40,
  56,
  33,
  51,
  23,
  41,
  38,
  17,
  49,
  23,
  35,
  29,
  29,
  33,
  67,
  45,
  56,
  54,
  55,
  33,
  26,
  39,
  45,
  60,
  18,
  27,
  68,
  33,
  38,
  20,
  59,
  39,
  61,
  51,
  25,
  76,
  67,
  37,
  41,
  49,
  43,
  37,
  58,
  35,
  29,
  25,
  24,
  51,
  38,
  19,
  45,
  53,
  23,
  33,
  62,
  22,
  54,
  31,
  23,
  56,
  30,
  23,
  61,
  66,
  53,
  52,
  80,
  58,
  40,
  37,
  73,
  60,
  80,
  42,
  29,
  43,
  68,
  53,
  53,
  30,
  51,
  50,
  62,
  33,
  18,
  49,
  27,
  41,
  47,
  48,
  56,
  49,
  38,
  62,
  52,
  43,
  40,
  28,
  67,
  58,
  28,
  36,
  50,
  77,
  50,
  17,
  25,
  24,
  25,
  30

In [8]:
df = client.query('SELECT * FROM bank.salaryincome')
df.result_rows

[(26, 3, 297991, 9, 13, 2, 11, 1, 1, 0, 0, 1977, 75, 0),
 (37, 3, 191342, 3, 2, 2, 2, 0, 1, 1, 0, 0, 40, 0),
 (18, 4294967295, 261276, 15, 10, 4, 4294967295, 3, 2, 0, 0, 1602, 40, 0),
 (28, 3, 149769, 11, 9, 4, 6, 1, 1, 1, 0, 0, 40, 0),
 (42, 5, 303044, 11, 9, 2, 4, 0, 1, 1, 0, 0, 40, 0),
 (27, 3, 292472, 15, 10, 2, 9, 0, 1, 1, 0, 0, 40, 0),
 (28, 3, 252424, 8, 11, 4, 13, 3, 2, 1, 0, 0, 40, 0),
 (51, 3, 63685, 11, 9, 2, 11, 0, 1, 1, 0, 0, 50, 0),
 (46, 3, 104632, 11, 9, 2, 6, 0, 1, 1, 0, 0, 40, 0),
 (42, 5, 342634, 8, 11, 0, 2, 1, 4, 1, 0, 0, 40, 0),
 (48, 3, 350440, 15, 10, 2, 2, 2, 1, 1, 0, 0, 40, 0),
 (65, 3, 255386, 11, 9, 4, 2, 2, 1, 1, 0, 0, 40, 0),
 (27, 3, 292472, 15, 10, 4, 2, 1, 1, 1, 0, 1876, 45, 0),
 (37, 6, 67083, 15, 10, 2, 9, 2, 1, 1, 0, 0, 40, 0),
 (35, 3, 93034, 11, 9, 4, 6, 1, 1, 1, 0, 0, 40, 0),
 (28, 3, 129624, 9, 13, 2, 11, 0, 4, 1, 0, 0, 45, 0),
 (34, 0, 67083, 9, 13, 4, 3, 4, 1, 1, 1471, 0, 40, 0),
 (36, 3, 116138, 11, 9, 2, 9, 0, 1, 1, 0, 0, 40, 0),
 (50, 3, 135

In [9]:
result = client.query('DESCRIBE table bank.salaryincome')
column_names = result.result_columns[0]
column_names

['age',
 'education',
 'workclass',
 'fnlwgt',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country',
 'income_>50K']

In [10]:
df = client.query('SELECT * FROM bank.salaryincome')
df = pd.DataFrame(df.result_rows, columns=column_names)
df

,age,education,workclass,fnlwgt,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_>50K
0,26,3,297991,9,13,2,11,1,1,0,0,1977,75,0
1,37,3,191342,3,2,2,2,0,1,1,0,0,40,0
2,18,4294967295,261276,15,10,4,4294967295,3,2,0,0,1602,40,0
3,28,3,149769,11,9,4,6,1,1,1,0,0,40,0
4,42,5,303044,11,9,2,4,0,1,1,0,0,40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175823,45,5,163559,9,13,2,3,5,1,0,0,0,48,4294967295
175824,23,3,216672,9,13,4,3,3,4,1,0,0,30,4294967295
175825,29,3,144259,9,13,2,5,0,2,1,4386,0,80,4294967295
175826,54,4,304570,9,13,2,6,0,1,1,7688,0,40,4294967295


In [9]:
!pip install ydata-profiling

In [14]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 4.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 KB 2.9 MB/s eta 0:00:00


In [52]:
categorical_columns

[]

In [53]:
numerical_columns

['age',
 'education',
 'workclass',
 'fnlwgt',
 'marital_status',
 'occupation',
 'relationship',
 'race',
 'gender',
 'capital_gain',
 'capital_loss',
 'hours_per_week',
 'native_country']

In [3]:
# create a catboost pipline

from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 
from sklearn.model_selection import cross_val_score
import sklearn
sklearn.set_config(transform_output="pandas")

#df = pd.read_csv('files/train.csv', low_memory=False)
df = pd.read_csv('/opt/spark/work-dir/files/train.csv')
# Define the features and target
y = df['income_>50K']
X = df.drop(columns=['income_>50K'], axis=1)

print(X.columns)


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the categorical and numerical columns
#categorical_columns = X.select_dtypes(include=['uint32']).columns
#numerical_columns = X.select_dtypes(include=['uint8']).columns
categorical_columns = list(X_train.select_dtypes(include=['object']).columns)
numerical_columns = list(X_train.select_dtypes(exclude=['object']).columns)

# Define the preprocessing steps
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    #('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns),
        ('cat', categorical_transformer, categorical_columns)
    ]
)

# Define the CatBoost model
model = CatBoostClassifier(iterations=1000,
                           learning_rate=0.1,
                           depth=6,
                           loss_function='Logloss',
                           verbose=False,
                           cat_features= ['cat__' + col for col in categorical_columns]
                           )

# Create the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Train the model
#pipeline.fit(X_train, y_train)

# Make predictions
#y_pred = pipeline.predict(X_test)

# Evaluate the model
#accuracy = accuracy_score(y_test, y_pred)
#print(f'Accuracy: {accuracy:.2f}')



Index(['age', 'workclass', 'fnlwgt', 'education', 'educational-num',
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')


In [4]:
counter = 0

In [5]:
counter += 1
def gridsearch_replacement(trial):
    params = {
        'model__iterations': trial.suggest_int('model__iterations', 100, 1000),
        'model__learning_rate': trial.suggest_float('model__learning_rate', 0.01, 0.3),
        'model__depth': trial.suggest_int('model__depth', 4, 10),
        'model__l2_leaf_reg': trial.suggest_float('model__l2_leaf_reg', 1, 10),
    }

    pipeline.set_params(**params)
    score = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='f1_macro').mean()
    return score

# Создаем исследование для оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(gridsearch_replacement, n_trials=4)

[I 2024-08-03 08:51:10,665] A new study created in memory with name: no-name-1373af98-31ee-471f-a1a9-5a1cd33d4d1c
[I 2024-08-03 08:52:01,659] Trial 0 finished with value: 0.8147957152363995 and parameters: {'model__iterations': 665, 'model__learning_rate': 0.16040999099916697, 'model__depth': 5, 'model__l2_leaf_reg': 1.1656395756302587}. Best is trial 0 with value: 0.8147957152363995.
[I 2024-08-03 08:52:49,414] Trial 1 finished with value: 0.8100783612045135 and parameters: {'model__iterations': 731, 'model__learning_rate': 0.2839439287035563, 'model__depth': 6, 'model__l2_leaf_reg': 8.046761133151414}. Best is trial 0 with value: 0.8147957152363995.
[I 2024-08-03 08:54:14,646] Trial 2 finished with value: 0.8070173251032499 and parameters: {'model__iterations': 502, 'model__learning_rate': 0.1427919497453483, 'model__depth': 10, 'model__l2_leaf_reg': 3.869451109950513}. Best is trial 0 with value: 0.8147957152363995.
[I 2024-08-03 08:54:53,174] Trial 3 finished with value: 0.81524608

In [6]:
y_probs = pipeline.predict_proba(X_test)[:, 1]

pred_test = y_probs > 0.5

f1_test = f1_score(pred_test, y_test, average='macro')


f1_test_1 = f1_score(pred_test, y_test, average='micro')


f1_test_2 = f1_score(pred_test, y_test, average='weighted')

accuracy = accuracy_score(pred_test, y_test)

precision = precision_score(pred_test, y_test, average='macro')

recall = recall_score(pred_test, y_test, average='macro')






fpr, tpr, thresholds = roc_curve(y_test, y_probs)
auc = roc_auc_score(y_test, y_probs)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc='lower right')
plt.show()

cm = confusion_matrix(y_test, pred_test)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False, xticklabels=['Predicted Negative', 'Predicted Positive'], yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

print(f'''
f1_test_macro: {f1_test}

f1_test_micro: {f1_test_1}

f1_test_weighted: {f1_test_2}

precision: {precision}

recall: {recall}

accuracy: {accuracy}

''')

#print('f1_test_macro:', f1_test, 'f1_test_micro:', f1_test_1, 'f1_test_weighted:', f1_test_2, 'accuracy:', accuracy, 'precision:', precision, 'recall:', recall)

y_test.value_counts(normalize=True)

un = pd.DataFrame({'y_test': y_test, 'pred_test': pred_test})
un['pred_test'] = un['pred_test'].astype('int')
un.value_counts(normalize=True)


NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
study.best_value

In [ ]:
study.best_params

In [ ]:
# Проверьте, существует ли эксперимент
experiment_name = "Catboost2"
experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment is None:
    # Создайте эксперимент, если он не существует
    experiment_id = mlflow.create_experiment(experiment_name)
else:
    # Используйте существующий эксперимент
    experiment_id = experiment.experiment_id

with mlflow.start_run(experiment_id=experiment_id, run_name=f'{counter}_experiment'):
    st_b = study.best_params
    mlflow.log_params(st_b)
    stud_bv = study.best_value

    pipeline.set_params(**st_b)
    pipeline.fit(X_train, y_train)
    mlflow.sklearn.log_model(pipeline, 'best_model')

    pred_test = pipeline.predict(X_test)
    f1_test = f1_score(pred_test, y_test, average='macro')

    log_sum = {'f1_macro_cv5': stud_bv, 'f1_score_macro': f1_test}
    mlflow.log_metrics(log_sum)


In [ ]:
stud_bv

In [ ]:
st_b

In [ ]:
erorr_xtest= f1_score(pred_test, y_test)
erorr_xtest

In [ ]:
import fastapi
fastapi.__version__

In [ ]:
import mlflow
logged_model = 'runs:/7db17d0e5d9140cfaa2afcf68b75f1b4/best_model'

# Load model as a PyFuncModel.
loaded_model = mlflow.sklearn.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(pd.DataFrame(df))

In [ ]:
dump(pipeline, 'bestmodel.joblib')

In [ ]:
joblib.load('bestmodel.joblib')

In [24]:
loaded_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'educational-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'gender',
                                                   'native-country'])])),
                ('model',
                 <catboost.core.CatBoostClassifier object at 0x7f94f855f4f0>)])

In [ ]:
X_test.to_csv('X_test.csv', index=False)
X_test

In [26]:
with open('X_test.csv','rb') as file:
    files={'file':file}
    response=requests.post(url='http://127.0.0.1:8004/predict', files=files)
response

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8004): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f94f8229ab0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [20]:
['cat__' + col for col in categorical_columns]
    

['cat__workclass',
 'cat__education',
 'cat__marital-status',
 'cat__occupation',
 'cat__relationship',
 'cat__race',
 'cat__gender',
 'cat__native-country']

'0.104.1'